# Databases

## Tables Description

- Table with user data:
    - id - unique user identifier (primary key)
    - gender - gender
    - age - age
    - country - country
    - city - city
    - exp_group - experimental group
    - os - operating system
    - source - traffic source  
<br></br>    
- Table with post data:
    - id - unique post identifier (primary key)
    - text - post text
    - topic - post topic  
<br></br>    
- Table with user action data (feed action):
    - user_id (→) user (id) - user identifier
    - post_id (→) post (id) - post identifier
    - action - action taken in the network
    - time - action time

## Tasks

### Task 1. 



Get data from a `user` table. 

In [1]:
import pandas as pd

In [2]:
connection = "postgresql:XXXXX"

In [3]:
request = '''
SELECT 
  * 
FROM 
  "user"
'''

In [4]:
df = pd.read_sql(request, connection)

df.head()

,id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


### Task 2. 

Get distinct values of topics from a `post` table. 

In [5]:
request = '''
SELECT
  DISTINCT(topic)
FROM
  "post"
'''

In [6]:
df = pd.read_sql(request, connection)

df

,topic
0,tech
1,covid
2,movie
3,entertainment
4,politics
5,sport
6,business


### Task 3. 

Get users older 30 years having iOS. 

In [7]:
request = '''
SELECT
  *
FROM
  "user"
WHERE
  age > 30
    AND os = 'iOS'
'''

In [8]:
df = pd.read_sql(request, connection)

df.head()

,id,gender,age,country,city,exp_group,os,source
0,212,1,47,Russia,Podolsk,0,iOS,ads
1,222,0,45,Russia,Ulyanovsk,0,iOS,ads
2,224,1,32,Russia,Yekaterinburg,2,iOS,ads
3,226,1,31,Russia,Moscow,4,iOS,ads
4,243,1,38,Kazakhstan,Ekibastuz,1,iOS,ads


### Task 4. 

Get users outside of Russia and they are nor in exp_group 0, nor in exp_group 3 or they are from Minsk. 

In [9]:
request = '''
SELECT
  *
FROM
  "user"
WHERE
  country <> 'Russia'
    AND (exp_group NOT IN (0, 3) OR city = 'Minsk')
'''

In [10]:
df = pd.read_sql(request, connection)

df.head()

,id,gender,age,country,city,exp_group,os,source
0,211,1,22,Belarus,Minsk,0,iOS,ads
1,213,1,22,Ukraine,Kamianske,2,iOS,ads
2,216,0,38,Ukraine,Lviv,1,Android,ads
3,218,1,25,Belarus,Minsk,2,iOS,ads
4,228,0,17,Azerbaijan,Baku,2,iOS,ads


### Task 5. 

What is an average age for each country?

In [11]:
request = '''
SELECT
  country,
  ROUND(AVG(age), 2) AS avg_age
FROM
  "user"
GROUP BY
  country
'''

In [12]:
df = pd.read_sql(request, connection)

df.head()

,country,avg_age
0,Azerbaijan,26.78
1,Belarus,27.28
2,Cyprus,28.07
3,Estonia,28.15
4,Finland,27.13


### Task 6. 

Aggregate data by `exp_group` and `os` and count total number of users, their min age and max age. 

In [13]:
request = '''
SELECT
  exp_group,
  os,
  COUNT(DISTINCT(id)) AS total_users,
  MAX(age) AS max_age,
  MIN(age) AS min_age
FROM
  "user"
GROUP BY
  exp_group, 
  os
'''

In [14]:
df = pd.read_sql(request, connection)

df.head()

,exp_group,os,total_users,max_age,min_age
0,0,Android,21234,95,14
1,0,iOS,11489,79,14
2,1,Android,21232,92,14
3,1,iOS,11406,87,14
4,2,Android,21102,78,14


In [15]:
df.to_csv('06_total_users.csv', sep=',', index=False)

### Task 7. 

Count max length of a text for each topic. 

In [16]:
request = '''
SELECT
  topic,
  MAX(LENGTH(text)) AS max_len
FROM
  "post"
GROUP BY
  topic
'''

In [17]:
df = pd.read_sql(request, connection)

df.head()

,topic,max_len
0,tech,16063
1,covid,148
2,movie,10234
3,entertainment,19004
4,politics,25392


### Task 8. 

Find a country having the least amount of users among countries having more than 1000 users. 

In [18]:
request = '''
SELECT
  country,
  COUNT(DISTINCT(id)) AS total_users
FROM
  "user"
GROUP BY
  country
HAVING
  COUNT(DISTINCT(id)) > 1000
ORDER BY 
  2
'''

In [19]:
df = pd.read_sql(request, connection)

df.head()

,country,total_users
0,Azerbaijan,1542
1,Finland,1599
2,Turkey,1606
3,Kazakhstan,3172
4,Belarus,3293


### Task 9. 

Count a number of users in each exp_group by the country. 

In [20]:
request = '''
SELECT 
  country, 
  exp_group, 
  COUNT(id) AS total_users
FROM 
  "user"
GROUP BY 
  country, 
  exp_group
'''

In [21]:
df = pd.read_sql(request, connection)

df.head()

,country,exp_group,total_users
0,Azerbaijan,0,341
1,Azerbaijan,1,271
2,Azerbaijan,2,327
3,Azerbaijan,3,308
4,Azerbaijan,4,295


### Task 10. 

Find exp_group size of users living in Moscow for groups average age of users in which is greater than 27.2 years. 

In [22]:
request = '''
SELECT 
  exp_group, 
  COUNT(DISTINCT(id)) AS total_users
FROM 
  "user"
WHERE
  city = 'Moscow'
GROUP BY 
  exp_group
HAVING
  AVG(age) > 27.2
'''

In [23]:
df = pd.read_sql(request, connection)

df.head()

,exp_group,total_users
0,1,4414
1,3,4378


### Task 11. 

Find the 3 top most popular topics. 

In [24]:
request = '''
SELECT
  topic,
  COUNT(DISTINCT(id)) AS posts_cnt
FROM
  "post"
GROUP BY
  topic
ORDER BY
  2 DESC
LIMIT
  3
'''

In [25]:
df = pd.read_sql(request, connection)

df.head()

,topic,posts_cnt
0,movie,3000
1,covid,1799
2,business,510


### Task 12. 

Get users from Voronezh and sort them by age in descending order. In case someone has the same age sort them by exp_group number in ascending order. 

In [26]:
request = '''
SELECT 
  *
FROM 
  "user"
WHERE
  city = 'Voronezh'
ORDER BY
  age DESC, 
  exp_group
'''

In [27]:
df = pd.read_sql(request, connection)

df.head()

,id,gender,age,country,city,exp_group,os,source
0,89990,0,71,Russia,Voronezh,0,Android,ads
1,2441,1,71,Russia,Voronezh,3,Android,ads
2,14184,1,66,Russia,Voronezh,3,Android,ads
3,35093,0,65,Russia,Voronezh,0,Android,ads
4,77284,0,64,Russia,Voronezh,4,Android,ads


### Task 13. 

Select post_id, time, age, and os for all likes from users in Omsk. Sort the results in descending order of time and save the first 100 values to a CSV format.

In [28]:
request = '''
SELECT 
  fa.post_id,
  fa.time,
  u.age,
  u.os
FROM 
  "user" u 
JOIN
  (
   SELECT
     f.post_id,
     f.user_id,
     f.time
   FROM
     "feed_action" f
   WHERE
     f.action = 'like'
  ) fa ON fa.user_id = u.id
WHERE
  u.city = 'Omsk'
ORDER BY
  2 DESC
LIMIT 
  100
'''

In [29]:
df = pd.read_sql(request, connection)

df.head()

,post_id,time,age,os
0,7159,2022-01-26 23:59:54,17,Android
1,7184,2022-01-26 23:56:30,17,Android
2,7265,2022-01-26 23:56:03,17,Android
3,7256,2022-01-26 23:55:24,17,Android
4,7080,2022-01-26 23:55:12,17,Android


In [30]:
df.shape

(100, 4)

In [31]:
df.to_csv('06_like_from_omsk.csv', sep=',', index=False)

### Task 14. 

Find the city where users aged 36 viewed the COVID-related post the least number of times on December 1, 2021, compared to other cities.Find the city where users aged 36 viewed the COVID-related post the least number of times on December 1, 2021, compared to other cities.

In [32]:
request = '''
WITH posts AS (
SELECT 
  f.user_id, 
  f.post_id
FROM 
  "feed_action" f
WHERE 
  f.action = 'view'
    AND DATE(f.time) = '2021-12-01'
),
topics AS (
SELECT 
  p.id
FROM 
  "post" p
WHERE 
  p.topic = 'covid'
),
old_users AS (
SELECT 
  u.id,
  u.city
FROM 
  "user" u 
WHERE
  u.age = 36
)

SELECT
  ou.city
FROM 
  old_users ou
JOIN
  posts po ON po.user_id = ou.id
JOIN
  topics t ON t.id = po.post_id
'''

In [33]:
df = pd.read_sql(request, connection)

df.head()

,city
0,Tula
1,Kolomna
2,Kolomna
3,Novorossiysk
4,Novorossiysk


In [34]:
df.city.value_counts().tail(1)

Arkhangelsk    5
Name: city, dtype: int64

### Task 15. 

Write an SQL query that retrieves: the top N posts by likes, the number of users who liked each post, the time of the most recent like.

In [35]:
request = '''
WITH most_likes AS (
SELECT 
  f.post_id,
  COUNT(f.user_id) AS likes
FROM 
  "feed_action" f
WHERE 
  f.action = 'like'
GROUP BY
  f.post_id
)

SELECT
  ml.*,
  mt.last_like_time
FROM
  most_likes ml
JOIN
  (
   SELECT
     post_id,
     MAX(time) as last_like_time
   FROM
     "feed_action" fa
   WHERE 
     fa.action = 'like'
   GROUP BY
     fa.post_id
  ) mt ON mt.post_id = ml.post_id 
ORDER BY
  2 DESC
'''

In [36]:
df = pd.read_sql(request, connection)

df.head()

,post_id,likes,last_like_time
0,4866,31714,2022-01-01 13:57:43
1,5127,29637,2022-01-05 13:52:17
2,6448,27939,2022-01-22 05:50:39
3,6498,27729,2022-01-22 21:45:11
4,6513,27686,2022-01-22 19:59:52
